### Generate ground truth results for T2B prompts 

In [1]:
import basico

c:\code\benchmark\AIAgents4Pharma\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Set 1

##### Precision Medicine Model

In [2]:
# generate expected answers for the basic prompts
# precision medicine model
#mod_precision = basico.load_biomodel('BIOMD0000000537')
mod_precision = basico.load_model(r'./models/BIOMD0000000537_url.xml')
print(basico.get_model_name())
# run simulation for set 1 and get the ground truth answer
tc = basico.run_time_course(2016, 2016)
tc.iloc[-1]["CRP{serum}"]

Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6R Antibody


2.269132134353497

##### Epidemiology Model

In [3]:
# epidemiology mode
#mod_epidemiology = basico.load_biomodel('BIOMD0000000971')
mod_epidemiology = basico.load_model(r'./models/Tang2020.xml')
print(basico.get_model_name())

Tang2020 - Estimation of transmission risk of COVID-19 and impact of public health interventions


##### Theoritical Model

In [4]:
# epidemiology mode
#mod_theory = basico.load_biomodel('BIOMD0000000027')
mod_theory = basico.load_model(r'./models/BIOMD0000000027_url.xml')
print(basico.get_model_name())

Markevich2004 - MAPK double phosphorylation,  ordered Michaelis-Menton


### Set 2 vary simulation time, interval and species to be simulated

In [5]:
def run_precision_model(interval, time, species = species, concentration = concentration):
    basico.set_species(name = species, model = mod_precision, initial_concentration = concentration)
    # check if the species was set correctly
    #print(basico.get_species(name = species, model = mod_precision)['initial_concentration'])
    # run simulations
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_precision, update_model = False)
    return tc.iloc[-1][["CRP{serum}",  "CRP{liver}", "IL6{serum}", "STAT3{gut}"]]

#run_precision_model(time = 100, interval = 2016)


NameError: name 'species' is not defined

In [ ]:
def run_epidemiology_model(interval, time):
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_epidemiology, update_model = False)
    return tc.iloc[-1][["Infected", "Susceptible", "Recovered", "Hospitalised"]]


In [ ]:
def run_theory_model(interval, time):
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_theory, update_model = False)
    return tc.iloc[-1][["Mpp", "M", "Mp"]]


In [ ]:
def run_theory_model_set_parameters(interval, time, parameter, value):
    basico.set_parameters(name = parameter, initial_value = value, model = mod_theory)
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_theory, update_model = False)
    return tc.iloc[-1][["Mpp", "M", "Mp"]]

In [ ]:
run_theory_model_set_parameters(1000, 1000, "Values[k1cat]", 0.1)

Mpp    499.080250
M        0.113662
Mp       0.806087
Name: 1000.0, dtype: float64

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# adjust initial concentrations of species
initial_concentr_CRP_serum = basico.get_species(name = "CRP{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_CRP_liver = basico.get_species(name = "CRP{liver}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_IL6_serum = basico.get_species(name = "IL6{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_STAT3_gut = basico.get_species(name = "STAT3{gut}", model = mod_precision)['initial_concentration'].iloc[0]
# time 
conditions = [
    (2016, 2016, "CRP{serum}", 0.01),
    (20, 100, "CRP{serum}", 1000),
    (2016, 4032, "CRP{serum}", 2.6),
    (1000, 1000, "IL6{serum}", initial_concentr_IL6_serum*10), 
    (2016, 1000, "CRP{liver}", initial_concentr_CRP_liver*10),
    (500, 500, "STAT3{gut}", initial_concentr_STAT3_gut*10),
    (1500, 2000, "STAT3{gut}", initial_concentr_STAT3_gut*0.10)
]

for time, interval, species, concentration in conditions:
    key = f"interval_{interval}_time_{time}_species_{species}_concentration_{concentration}"
    print (concentration)
    results[key] = run_precision_model(interval, time, species, concentration)

results

0.01
1000
2.6
435628.8965511659
1583.2584678161063
6.106360135082123e-08
6.106360135082122e-10


{'interval_2016_time_2016_species_CRP{serum}_concentration_0.01': CRP{serum}    10.301993
 CRP{liver}     6.707914
 IL6{serum}     0.000641
 STAT3{gut}     9.124980
 Name: 2016.0, dtype: float64,
 'interval_100_time_20_species_CRP{serum}_concentration_1000': CRP{serum}    243.477247
 CRP{liver}    162.273954
 IL6{serum}      0.003962
 STAT3{gut}      0.291445
 Name: 20.0, dtype: float64,
 'interval_4032_time_2016_species_CRP{serum}_concentration_2.6': CRP{serum}    10.302718
 CRP{liver}     6.708469
 IL6{serum}     0.000641
 STAT3{gut}     9.124957
 Name: 2016.0, dtype: float64,
 'interval_1000_time_1000_species_IL6{serum}_concentration_435628.8965511659': CRP{serum}    91.299723
 CRP{liver}    60.894795
 IL6{serum}     0.000642
 STAT3{gut}     6.813039
 Name: 1000.0, dtype: float64,
 'interval_1000_time_2016_species_CRP{liver}_concentration_1583.2584678161063': CRP{serum}    10.328675
 CRP{liver}     6.728273
 IL6{serum}     0.000641
 STAT3{gut}     9.124149
 Name: 2016.0, dtype: floa

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (50, 100),
    (100, 200),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_epidemiology_model(interval, time)

results

{'interval_100_time_50': Infected        1.043385e+05
 Susceptible     1.017891e+06
 Recovered       2.231583e+06
 Hospitalised    1.325140e+05
 Name: 50.0, dtype: float64,
 'interval_200_time_100': Infected        7.143353e+04
 Susceptible     1.055609e+06
 Recovered       4.586298e+06
 Hospitalised    8.688262e+04
 Name: 100.0, dtype: float64,
 'interval_400_time_180': Infected        4.031662e+04
 Susceptible     1.055585e+06
 Recovered       6.955857e+06
 Hospitalised    4.901975e+04
 Name: 180.0, dtype: float64,
 'interval_1000_time_500': Infected        4.090069e+03
 Susceptible     1.055585e+06
 Recovered       9.714202e+06
 Hospitalised    4.972991e+03
 Name: 500.0, dtype: float64,
 'interval_400_time_20': Infected        1.206681e+05
 Susceptible     3.167693e+06
 Recovered       1.634611e+05
 Hospitalised    4.076004e+04
 Name: 20.0, dtype: float64,
 'interval_10_time_10': Infected        3.413654e+03
 Susceptible     1.079620e+07
 Recovered       2.980390e+03
 Hospitalised  

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (1000, 100),
    (1000, 1000),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_theory_model(interval, time)

results

{'interval_100_time_1000': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 1000.0, dtype: float64,
 'interval_1000_time_1000': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 1000.0, dtype: float64,
 'interval_400_time_180': Mpp    491.137882
 M        3.033640
 Mp       5.828478
 Name: 180.0, dtype: float64,
 'interval_1000_time_500': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 500.0, dtype: float64,
 'interval_400_time_20': Mpp     45.234214
 M      424.991709
 Mp      29.774077
 Name: 20.0, dtype: float64,
 'interval_10_time_10': Mpp     17.029310
 M      460.393300
 Mp      22.577389
 Name: 10.0, dtype: float64}

In [ ]:
# run theory model with different parameters
results = {}
# time 
interval = 1000
conditions = [
    (100, 'k1cat', 0.1),
    (1000, 'k1cat', 1),
    (180, 'k1cat', 100),
    (500, 'k2cat', 10), 
    (20, 'k2cat', 0.1),
    (10, 'k2cat', 100)
]

for time, parameter, value in conditions:
    key = f"time_{time}_parameter_{parameter}_value_{value}"
    results[key] = run_theory_model_set_parameters(interval, time, parameter, value)

results

{'time_100_parameter_k1cat_value_0.1': Mpp    317.356267
 M      154.982325
 Mp      27.661408
 Name: 100.0, dtype: float64,
 'time_1000_parameter_k1cat_value_1': Mpp    494.565011
 M        0.076041
 Mp       5.358948
 Name: 1000.0, dtype: float64,
 'time_180_parameter_k1cat_value_100': Mpp    494.647985
 M        0.000758
 Mp       5.351257
 Name: 180.0, dtype: float64,
 'time_500_parameter_k2cat_value_10': Mpp    491.982136
 M        0.001140
 Mp       8.016723
 Name: 500.0, dtype: float64,
 'time_20_parameter_k2cat_value_0.1': Mpp     42.542954
 M        0.103107
 Mp     457.353939
 Name: 20.0, dtype: float64,
 'time_10_parameter_k2cat_value_100': Mpp    499.195565
 M        0.000113
 Mp       0.804322
 Name: 10.0, dtype: float64}

### Steady state

In [9]:
mod_precision = basico.load_model(r'./models/BIOMD0000000537_url.xml')
basico.task_steadystate.run_steadystate()
basico.get_species().head()

Error while running the simulation: >WARNING 2025-09-19T17:39:31<
  CSteadyState (2): No steady state was found.


,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
IL6,serum,reactions,nmol/l,0.000436,2.623419e+11,,,0.000436,2.623419e+11,-1.621620e-17,-0.009766,Metabolite_114,mwf626e95e_543f_41e4_aad4_c6bf60ab345b,0.028719,IL6{serum}
sgp130,serum,reactions,nmol/l,3.900000,2.348635e+15,,,3.900000,2.348635e+15,4.167564e-16,0.250977,Metabolite_115,mwbbbce920_e8dd_4320_9386_fc94bfb2fc99,0.997567,sgp130{serum}
sR_IL6_sgp130,serum,reactions,nmol/l,0.087406,5.263716e+13,,,0.087406,5.263716e+13,-2.367565e-16,-0.142578,Metabolite_116,mw810ff751_fa4e_4143_bd50_169b3e325e1e,9.190533,sR_IL6_sgp130{serum}
CRP,serum,reactions,nmol/l,221.063676,1.331277e+17,,,221.063676,1.331277e+17,6.243627e-13,376.000000,Metabolite_117,mw114aa90f_5f5b_4fe8_9406_361c8489b6a1,2.777778,CRP{serum}
sR,serum,reactions,nmol/l,4.253507,2.561522e+15,,,4.253507,2.561522e+15,1.328431e-14,8.000000,Metabolite_118,mw30ae63db_6cd3_4b6f_93ad_3350cd360bcc,3.283772,sR{serum}


In [10]:
mod_theory = basico.load_model(r'./models/BIOMD0000000027_url.xml')
basico.task_steadystate.run_steadystate()
basico.get_species()

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
M,uVol,reactions,nmol/l,500.0,3.011070e+17,,,437.729984,2.636072e+17,1.037837e-16,0.0625,Metabolite_158,M,978.030464,M
Mp,uVol,reactions,nmol/l,0.0,0.000000e+00,,,12.852481,7.739945e+15,-4.151348e-16,-0.2500,Metabolite_159,Mp,5.313708,Mp
Mpp,uVol,reactions,nmol/l,0.0,0.000000e+00,,,49.417535,2.975994e+16,4.151348e-16,0.2500,Metabolite_160,Mpp,25.070051,Mpp
MAPKK,uVol,fixed,nmol/l,50.0,3.011070e+16,,,50.000000,3.011070e+16,0.000000e+00,0.0000,Metabolite_161,MAPKK,NaN,MAPKK
MKP3,uVol,fixed,nmol/l,100.0,6.022141e+16,,,100.000000,6.022141e+16,0.000000e+00,0.0000,Metabolite_162,MKP3,NaN,MKP3


In [11]:
mod_epidemiology = basico.load_model(r'./models/Tang2020.xml')
basico.task_steadystate.run_steadystate()
basico.get_species()

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
Susceptible,Wuhan,reactions,#/ml,1.108100e+07,1.108100e+07,,,1.107825e+07,1.107825e+07,4.237148e-24,4.237148e-24,Metabolite_163,Susceptible,9.765576e+29,Susceptible
Exposed,Wuhan,reactions,#/ml,1.051000e+02,1.051000e+02,,,5.997059e-24,5.997059e-24,-1.452434e-25,-1.452434e-25,Metabolite_164,Exposed,7.000007e+00,Exposed
Infected,Wuhan,reactions,#/ml,2.767900e+01,2.767900e+01,,,1.137373e-24,1.137373e-24,2.173586e-25,2.173586e-25,Metabolite_165,Infected,1.528880e+00,Infected
Asymptomatic,Wuhan,reactions,#/ml,5.353900e+01,5.353900e+01,,,-9.305782e-25,-9.305782e-25,2.428722e-25,2.428722e-25,Metabolite_166,Asymptomatic,3.831555e+00,Asymptomatic
Susceptible_quarantined,Wuhan,reactions,#/ml,7.390000e+02,7.390000e+02,,,1.588187e-22,1.588187e-22,-4.948627e-24,-4.948627e-24,Metabolite_167,Susceptible_quarantined,1.400001e+01,Susceptible_quarantined
Exposed_quarantined,Wuhan,reactions,#/ml,1.164200e+00,1.164200e+00,,,-1.085675e-24,-1.085675e-24,1.366866e-25,1.366866e-25,Metabolite_168,Exposed_quarantined,7.942804e+00,Exposed_quarantined
Hospitalised,Wuhan,reactions,#/ml,1.000000e+00,1.000000e+00,,,-1.240771e-24,-1.240771e-24,1.584469e-25,1.584469e-25,Metabolite_169,Hospitalised,4.204104e+00,Hospitalised
Recovered,Wuhan,reactions,#/ml,2.000000e+00,2.000000e+00,,,2.030779e+01,2.030779e+01,1.013596e-25,1.013596e-25,Metabolite_170,Recovered,5.405853e+25,Recovered


#### Search models


In [11]:
from basico import biomodels

In [15]:
# Search model ussing basico
results = biomodels.search_for_model(query = "precision medicine",
                                      num_results = 100)
print(len(results))

results

7


[{'format': 'SBML',
  'id': 'BIOMD0000000583',
  'lastModified': None,
  'name': 'Leber2015 - Mucosal immunity and gut microbiome interaction during C. difficile infection',
  'submissionDate': '2015-07-19T23:00:00Z',
  'submitter': 'Andrew Leber',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000583'},
 {'format': 'SBML',
  'id': 'BIOMD0000000962',
  'lastModified': None,
  'name': 'Zhao2020 - SUQC model of COVID-19 transmission dynamics in Wuhan, Hubei, and China',
  'submissionDate': '2020-08-25T23:00:00Z',
  'submitter': 'Kausthubh Ramachandran',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000962'},
 {'format': 'SBML',
  'id': 'MODEL2106070001',
  'lastModified': None,
  'name': 'Montagud2022 - Prostate cancer Boolean model',
  'submissionDate': '2021-06-06T23:00:00Z',
  'submitter': 'Arnau Montagud',
  'url': 'https://www.ebi.ac.uk/biomodels/MODEL2106070001'},
 {'format': 'SBML',
  'id': 'MODEL2203300001',
  'lastModified': None,
  'name': 'Bannerman2022 Whole Genome M

In [ ]:
results = biomodels.search_for_model(query = "crohn's disease",
                                      num_results = 100)
print(len(results))
results

5


[{'format': 'SBML',
  'id': 'BIOMD0000000534',
  'lastModified': None,
  'name': 'Dwivedi2014 - Healthy Volunteer IL6 Model',
  'submissionDate': '2014-08-04T23:00:00Z',
  'submitter': 'Vincent Knight-Schrijver',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000534'},
 {'format': 'SBML',
  'id': 'BIOMD0000000535',
  'lastModified': None,
  'name': 'Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6 Antibody',
  'submissionDate': '2014-08-04T23:00:00Z',
  'submitter': 'Vincent Knight-Schrijver',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000535'},
 {'format': 'SBML',
  'id': 'BIOMD0000000536',
  'lastModified': None,
  'name': 'Dwivedi2014 - Crohns IL6 Disease model - sgp130 activity',
  'submissionDate': '2014-08-04T23:00:00Z',
  'submitter': 'Vincent Knight-Schrijver',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000536'},
 {'format': 'SBML',
  'id': 'BIOMD0000000537',
  'lastModified': None,
  'name': 'Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6R Antibody',
  

In [17]:
results = biomodels.search_for_model(query = "covid-19",
                                      num_results = 100)
print(len(results))
results

43


[{'format': 'SBML',
  'id': 'BIOMD0000000955',
  'lastModified': None,
  'name': 'Giordano2020 - SIDARTHE model of COVID-19 spread in Italy',
  'submissionDate': '2020-07-27T23:00:00Z',
  'submitter': 'Kausthubh Ramachandran',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000955'},
 {'format': 'SBML',
  'id': 'BIOMD0000000956',
  'lastModified': None,
  'name': 'Bertozzi2020 - SIR model of scenarios of COVID-19 spread in CA and NY',
  'submissionDate': '2020-08-06T23:00:00Z',
  'submitter': 'Kausthubh Ramachandran',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000956'},
 {'format': 'SBML',
  'id': 'BIOMD0000000957',
  'lastModified': None,
  'name': 'Roda2020 - SIR model of COVID-19 spread in Wuhan',
  'submissionDate': '2020-08-10T23:00:00Z',
  'submitter': 'Kausthubh Ramachandran',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000957'},
 {'format': 'SBML',
  'id': 'BIOMD0000000958',
  'lastModified': None,
  'name': 'Ndairou2020 - early-stage transmission dynamics of 

In [18]:
results = biomodels.search_for_model(query = "gut",
                                      num_results = 100)
print(len(results))
results

58


[{'format': 'SBML',
  'id': 'BIOMD0000000451',
  'lastModified': None,
  'name': 'Carbo2013 - Cytokine driven CD4+ T Cell differentiation and phenotype plasticity',
  'submissionDate': '2013-04-22T23:00:00Z',
  'submitter': 'Adria Carbo',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000451'},
 {'format': 'SBML',
  'id': 'BIOMD0000000527',
  'lastModified': None,
  'name': 'Kaiser2014 - Salmonella persistence after ciprofloxacin treatment',
  'submissionDate': '2013-12-17T00:00:00Z',
  'submitter': 'Roland Regoes',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000527'},
 {'format': 'SBML',
  'id': 'BIOMD0000000583',
  'lastModified': None,
  'name': 'Leber2015 - Mucosal immunity and gut microbiome interaction during C. difficile infection',
  'submissionDate': '2015-07-19T23:00:00Z',
  'submitter': 'Andrew Leber',
  'url': 'https://www.ebi.ac.uk/biomodels/BIOMD0000000583'},
 {'format': 'SBML',
  'id': 'BIOMD0000000619',
  'lastModified': None,
  'name': 'Sluka2016  - Acetamin